In [100]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import os
import json

https://drive.google.com/file/d/1Pm9KebSPhC0hyoDl21Q7taOqDM9QpWT0/view?usp=sharing

In [101]:
def get_angle(edge1,  edge2, points):
    assert tuple(sorted(edge1)) in edges
    assert tuple(sorted(edge2)) in edges
    edge1 = set(edge1)
    edge2 = set(edge2)
    mid_point = edge1.intersection(edge2).pop()
    a = (edge1-edge2).pop()
    b = (edge2-edge1).pop()
    v1 = points[mid_point]-points[a]
    v2 = points[mid_point]-points[b]
    angle = (math.degrees(np.arccos(np.dot(v1[0:2],v2[0:2])/(np.linalg.norm(v1[0:2])*np.linalg.norm(v2[0:2])))))
    return angle
edges = {(0, 1), (0, 15), (0, 16), (1, 2), (1, 5), (1, 8), (2, 3), (3, 4), (5, 6), (6, 7), (8, 9), (8, 12), (9, 10), (10, 11), (11, 22), (11, 24), (12, 13), (13, 14), (14, 19), (14, 21), (15, 17), (16, 18), (19, 20), (22, 23)}

In [102]:
def icc(Y, icc_type='ICC(3,1)'):   
    [n, k] = Y.shape
    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()
    MSE = SSE / dfe

    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc  # / n (without n in SPSS results)

    SSR = SST - SSC - SSE
    MSR = SSR / dfr
    if icc_type == 'ICC(2,1)' or icc_type == 'ICC(2,k)':
        if icc_type == 'ICC(2,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)
    elif icc_type == 'ICC(3,1)' or icc_type == 'ICC(3,k)':
        if icc_type == 'ICC(3,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)
    return ICC

In [103]:
def calc_angles(coordinates):
  shoulder_angles = []
  trunk_angles = []
  for i in range(len(coordinates)):
    points = np.array(coordinates[i] ).reshape(-1,3)
    if(points[1][0]!=0 and points[2][0]!=0 and points[5][0]!=0 and points[8][0]!=0):
      x=get_angle((1,2), (1,5),points)
      if (x>100):
        shoulder_angles.append(x)
        y=get_angle((1,5), (1,8),points)
        trunk_angles.append(y) 
  return shoulder_angles,trunk_angles

In [196]:
def action_coord(action,dataset):
  coordinates=[]
  count=0
  with open(f'/content/drive/MyDrive/TinyVIRAT/tiny_'+dataset+'.json','r') as j:
      contents = json.loads(j.read())
      for i in range(len(contents['tubes'])):
        if action in contents['tubes'][i]['label']:
          L = [files for a,b,files in os.walk('/content/drive/MyDrive/PoseEstimation_TinyVirat/'+dataset+'/'+contents['tubes'][i]['video_id']+'/'+str(contents['tubes'][i]['id'])+'.json')]
          if len(L)!=0:  
            for j in range(1,4):
              with open(f'/content/drive/MyDrive/PoseEstimation_TinyVirat/'+dataset+'/'+contents['tubes'][i]['video_id']+'/'+str(contents['tubes'][i]['id'])+'.json/'+L[0][int(len(L[0])*j/4)],'r') as j2:
                contents2 = json.loads(j2.read())
                try:
                  coordinates.append(contents2["people"][0]["pose_keypoints_2d"])
                  count=count+1
                except:
                  pass
        if count >= 300:
          break 
  return coordinates,count

In [105]:
def icc_calc(activity,dataset):
  coordinates,count = action_coord(activity,dataset)
  shoulder_angles,trunk_angles = calc_angles(coordinates)
  knee_angles,hip_angles = calc_angles_leg(coordinates)
  datacount = min(len(shoulder_angles),len(knee_angles))
  shoulder_angles = shoulder_angles[:datacount]
  trunk_angles = trunk_angles[:datacount]
  knee_angles = knee_angles[:datacount]
  hip_angles = hip_angles[:datacount]
  shoulder_angles = np.array(shoulder_angles)
  trunk_angles = np.array(trunk_angles)
  knee_angles = np.array(knee_angles)
  hip_angles = np.array(hip_angles)
  arrayy = np.stack((shoulder_angles,trunk_angles,knee_angles,hip_angles), axis=0)
  return coordinates,icc(arrayy),len(shoulder_angles)/count


In [146]:
def reliability_vid(vidid,id): 
  coordinates = []
  L = [files for a,b,files in os.walk('/content/drive/MyDrive/PoseEstimation_TinyVirat/train/'+vidid+'/'+str(id)+'.json')]
  for j in range(len(L[0])):
      with open(f'/content/drive/MyDrive/PoseEstimation_TinyVirat/train/'+vidid+'/'+str(id)+'.json/'+L[0][j],'r') as j2:
          contents2 = json.loads(j2.read())
          try:
            coordinates.append(contents2["people"][0]["pose_keypoints_2d"])
          except:
            pass
  print (len(coordinates)/len(L[0]))

In [182]:
reliability_vid('VIRAT_S_000201_08_001652_001838',4186)

0.5714285714285714


In [ ]:
coordinates,icc_val,reliability = icc_calc('Misc','train')
print(icc_val,reliability)

In [7]:
def calc_angles_leg(coordinates):
  knee_angles = []
  hip_angles = []
  for i in range(len(coordinates)):
    points = np.array(coordinates[i] ).reshape(-1,3)
    if(points[12][0]!=0 and points[13][0]!=0 and points[14][0]!=0 and points[8][0]!=0):
      y=get_angle((12,13), (8,12),points)
      if(y>30):
        hip_angles.append(y)
        x=get_angle((12,13), (13,14),points)
        knee_angles.append(x)
  return knee_angles,hip_angles

In [113]:
pd.DataFrame(calc_angles_leg(coordinates)[0]).describe()

,0
count,118.000000
mean,169.372725
std,12.065604
min,127.028825
25%,167.358957
50%,174.065281
75%,176.674245
max,179.998011


In [ ]:
pd.DataFrame(trunk_angles).describe()

In [190]:
activity_set_test = set([])
activity_dict_test={}
with open(f'/content/drive/MyDrive/TinyVIRAT/tiny_test.json','r') as j:
    contents = json.loads(j.read())
    for i in range(len(contents['tubes'])):
      for j in range(len(contents['tubes'][i]['label'])):
        if(contents['tubes'][i]['label'][j] not in activity_dict_test):
          activity_dict_test[contents['tubes'][i]['label'][j]] = 0
        else:
          activity_dict_test[contents['tubes'][i]['label'][j]]+=1
        activity_set_test.add(contents['tubes'][i]['label'][j])
print(activity_dict_test)

{'Entering': 20, 'Closing': 11, 'activity_walking': 3273, 'vehicle_moving': 692, 'Exiting': 19, 'activity_standing': 945, 'vehicle_stopping': 82, 'activity_carrying': 1425, 'activity_running': 34, 'vehicle_turning_left': 37, 'specialized_talking_phone': 65, 'Opening': 19, 'vehicle_turning_right': 26, 'Interacts': 221, 'vehicle_starting': 55, 'Misc': 18, 'specialized_using_tool': 89, 'Loading': 2, 'Riding': 117, 'activity_gesturing': 14, 'Transport_HeavyCarry': 85, 'Push': 43, 'Pull': 41, 'specialized_miscellaneous': 9, 'Talking': 51, 'specialized_texting_phone': 6}


In [188]:
activity_set_train = set([])
activity_dict_train={}
with open(f'/content/drive/MyDrive/TinyVIRAT/tiny_train.json','r') as j:
    contents = json.loads(j.read())
    for i in range(len(contents['tubes'])):
      for j in range(len(contents['tubes'][i]['label'])):
        if(contents['tubes'][i]['label'][j] not in activity_dict_train):
          activity_dict_train[contents['tubes'][i]['label'][j]] = 0
        else:
          activity_dict_train[contents['tubes'][i]['label'][j]]+=1
        activity_set_train.add(contents['tubes'][i]['label'][j])
print(activity_dict_train)

{'Exiting': 31, 'activity_walking': 4643, 'vehicle_moving': 842, 'activity_carrying': 2560, 'vehicle_turning_left': 37, 'vehicle_stopping': 117, 'Talking': 92, 'activity_standing': 1830, 'Interacts': 287, 'Closing': 16, 'Opening': 25, 'vehicle_turning_right': 32, 'activity_gesturing': 28, 'Pull': 134, 'specialized_talking_phone': 59, 'Transport_HeavyCarry': 60, 'vehicle_starting': 71, 'Entering': 21, 'Loading': 9, 'specialized_miscellaneous': 13, 'specialized_texting_phone': 41, 'Riding': 69, 'specialized_using_tool': 189, 'activity_running': 32, 'Push': 12, 'Misc': 51}


In [197]:
icc_val={}
activity_list = list(activity_set_train)
for activity in activity_list:
  icc_val[activity] = icc_calc(activity,'train')
  print(activity, icc_val[activity])

vehicle_moving ([[0, 0, 0, 24.3949, 15.2888, 0.414736, 26.9733, 15.2926, 0.333945, 29.8102, 21.8309, 0.124646, 0, 0, 0, 21.5567, 15.0124, 0.341255, 20.1408, 20.7079, 0.075576, 0, 0, 0, 24.1246, 26.6749, 0.394603, 26.4074, 26.6805, 0.289484, 27.2452, 34.6508, 0.268164, 28.6744, 42.3196, 0.255043, 21.8481, 26.6791, 0.377505, 21.5573, 35.7964, 0.378809, 20.7087, 44.8789, 0.322888, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20.9875, 44.8983, 0.218306, 20.4228, 44.9031, 0.22864, 20.9848, 45.1794, 0.348177, 30.9369, 43.18, 0.107738, 0, 0, 0, 28.9529, 42.6202, 0.249212], [40.5888, 36.1604, 0.156302, 37.8831, 46.7388, 0.240006, 23.6214, 44.7734, 0.262519, 0, 0, 0, 0, 0, 0, 51.6569, 48.2131, 0.108301, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37.6398, 31.488, 0.168823, 44.5212, 33.7027, 0.176413, 33.7055, 32.7185, 0.206489, 48.2144, 35.1832, 0.118094, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 68.9147, 30.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


vehicle_turning_left ([[13.6607, 26.9876, 0.153702, 12.7161, 29.3685, 0.407645, 11.1193, 29.5257, 0.411986, 10.0141, 34.286, 0.385524, 13.6653, 35.5555, 0.205977, 14.9284, 29.5229, 0.35709, 15.4101, 33.1723, 0.113976, 0, 0, 0, 12.7151, 36.5004, 0.180367, 11.13, 36.5039, 0.198892, 0, 0, 0, 0, 0, 0, 14.6173, 36.1864, 0.154434, 0, 0, 0, 0, 0, 0, 12.7158, 26.5072, 0.173918, 13.8225, 26.5072, 0.143758, 12.0788, 26.5122, 0.176457, 14.2983, 26.6667, 0.0854219, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [48.6822, 49.0291, 0.109754, 56.2501, 56.9506, 0.249289, 49.0284, 59.0045, 0.145355, 0, 0, 0, 0, 0, 0, 62.1013, 55.9105, 0.473669, 59.6947, 70.3659, 0.367297, 47.9889, 71.0639, 0.249129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 47.3093, 47.6533, 0.0590539, 51.0879, 46.2718, 0.0962723, 0, 0, 0, 55.5653, 46.6146, 0.105844, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 45.6884, 73.7426, 0.615421, 57.2196, 74.3652, 0.551489, 61.2662, 81.2125, 0.1